In [1]:
#Install Package

# !pip install google-cloud
# !pip install google-cloud-pubsub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.3/321.3 kB 20.6 MB/s eta 0:00:00


In [2]:
#Importacion de librerias
import os
import requests
import json
from time import sleep
from google.cloud import pubsub_v1

In [3]:
#configurar las credenciales de GCP en el servidor
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'/content/steady-freedom-476101-u7-353c7a2ff153.json'

In [4]:
# TODO(developer)
#project_id = "your-project-id"
project_id = "steady-freedom-476101-u7"
# topic_id = "your-topic-id"
topic_id = "topico-vuelos"
publisher = pubsub_v1.PublisherClient()
project_path = f"projects/{project_id}"

In [5]:
# AVIATION API parameters
access_key = "b5eaefa067fbe65dae71f75d34f7b96f"
limit = "5"
flight_status = "active"
arr_iata = "BCN"

aviation_url = "http://api.aviationstack.com/v1/flights" + \
            "?access_key=" + access_key + \
            "&flight_status=" + flight_status + \
            "&arr_iata=" + arr_iata
            # "&limit=" + limit

topic_path = publisher.topic_path(project_id, topic_id)

In [7]:
# Main code
while True:
    # Get all flights
    response = requests.get(aviation_url)
    if response.status_code != 200:
        print("Algo ocurrió. ERROR: ", response.status_code)
        print(response.json)
        exit()

    json_text = response.text

    python_obj = json.loads(json_text)

    data_array = []
    for flight in python_obj['data']:
        data = {}

        # Only flight in the air
        if flight['live'] != None:
            data['vuelo'] = flight['flight']['iata']
            data['aerolinea'] = flight['airline']['name']
            data['fecha_vuelo'] = flight['flight_date']
            data['origen'] = flight['departure']['airport']
            data['destino'] = flight['arrival']['airport']
            data['latitud'] = flight['live']['latitude']
            data['longitud'] = flight['live']['longitude']
            data['altitud'] = flight['live']['altitude']
            data['velocidad'] = flight['live']['speed_horizontal']
            data['timestamp'] = flight['live']['updated']
            print("Enviando información del vuelo " + data['vuelo'])
            data_array.append(data)

    print("\nEnviando array: ")
    print(data_array)

    # When you publish a message, the client returns a future.
    future = publisher.publish(topic_path, json.dumps(data_array).encode("utf-8"))
    print(future.result())

    print("\n")

    # Wait till next refresh
    sleep(5)

Enviando información del vuelo SQ388
Enviando información del vuelo MU249
Enviando información del vuelo ZH865
Enviando información del vuelo CA845
Enviando información del vuelo EY111
Enviando información del vuelo QR137
Enviando información del vuelo 5X242
Enviando información del vuelo LA8114
Enviando información del vuelo AA112
Enviando información del vuelo DL194
Enviando información del vuelo UA992
Enviando información del vuelo UA120
Enviando información del vuelo AA742

Enviando array: 
[{'vuelo': 'SQ388', 'aerolinea': 'Singapore Airlines', 'fecha_vuelo': '2025-12-05', 'origen': 'Singapore Changi', 'destino': 'El Prat De Llobregat', 'latitud': 42.9274, 'longitud': 18.9758, 'altitud': 12192, 'velocidad': 879.7, 'timestamp': '2025-12-05T03:56:20+00:00'}, {'vuelo': 'MU249', 'aerolinea': 'China Eastern Airlines', 'fecha_vuelo': '2025-12-05', 'origen': 'Shanghai Pudong International', 'destino': 'El Prat De Llobregat', 'latitud': 53.7491, 'longitud': 15.7657, 'altitud': 11582.4, 've

KeyboardInterrupt: 